In [143]:
import pandas as pd

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

In [48]:
from sklearn import svm

In [2]:
df = pd.read_csv('../data/diamonds_train.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(5)

,carat,cut,color,clarity,depth,table,x,y,z,city,price
0,1.21,Premium,J,VS2,62.4,58.0,6.83,6.79,4.25,Dubai,4268
1,0.32,Very Good,H,VS2,63.0,57.0,4.35,4.38,2.75,Kimberly,505
2,0.71,Fair,G,VS1,65.5,55.0,5.62,5.53,3.65,Las Vegas,2686
3,0.41,Good,D,SI1,63.8,56.0,4.68,4.72,3.00,Kimberly,738
4,1.02,Ideal,G,SI1,60.5,59.0,6.55,6.51,3.95,Dubai,4882


In [3]:
clar_lab_mm = pd.read_csv('../data/scales/clar_lab_mm.csv')
cut_lab_mm = pd.read_csv('../data/scales/cut_lab_mm.csv')
col_lab_mm = pd.read_csv('../data/scales/col_lab_mm.csv')

# clar_lab_rob = pd.read_csv('../data/scales/clar_lab_mm.csv')
# cut_lab_rob = pd.read_csv('../data/scales/cut_lab_mm.csv')
# col_lab_rob = pd.read_csv('../data/scales/col_lab_mm.csv')

clar_lab_mm.drop(columns=['Unnamed: 0'], inplace=True)
cut_lab_mm.drop(columns=['Unnamed: 0'], inplace=True)
col_lab_mm.drop(columns=['Unnamed: 0'], inplace=True)

# clar_lab_rob.drop(columns=['Unnamed: 0'], inplace=True)
# cut_lab_rob.drop(columns=['Unnamed: 0'], inplace=True)
# col_lab_rob.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
df_2 = df.merge(clar_lab_mm[['clarity', 'label']])
# xx = df_2.merge(clar_lab_rob[['clarity', 'label']])


df_2.drop(columns='clarity', inplace=True)
df_2.rename(columns={'label': 'clarity'}, inplace=True)

In [5]:
df_2 = df_2.merge(cut_lab_mm[['cut', 'label']])
# df_2 = df_2.merge(cut_lab_rob[['cut', 'label']])
df_2.drop(columns='cut', inplace=True)
df_2.rename(columns={'label': 'cut'}, inplace=True)

df_2 = df_2.merge(col_lab_mm[['color', 'label']])
# df_2 = df_2.merge(col_lab_rob[['color', 'label']])
df_2.drop(columns='color', inplace=True)
df_2.rename(columns={'label': 'color'}, inplace=True)

In [6]:
df_2 = df_2.drop(columns='city')

In [7]:
df_2['xyz'] = df_2['x'] * df_2['y'] * df_2['z']
df_2.drop(columns=['x', 'y', 'z'], inplace=True)
df_2

,carat,depth,table,price,clarity,cut,color,xyz
0,1.21,62.4,58.0,4268,0.524187,0.604337,0.007570,197.096725
1,1.35,60.5,56.0,6254,0.524187,0.604337,0.007570,221.664824
2,1.12,58.6,58.0,4422,0.524187,0.604337,0.007570,186.594000
3,1.26,62.0,60.0,5173,0.524187,0.604337,0.007570,206.486000
4,1.20,60.8,60.0,4839,0.524187,0.604337,0.007570,197.281370
...,...,...,...,...,...,...,...,...
40450,0.70,65.9,57.0,1158,0.012153,0.130465,0.739817,113.843400
40451,0.70,66.5,62.0,992,0.012153,0.130465,0.739817,111.621582
40452,0.50,68.3,58.0,627,0.012153,0.130465,0.739817,77.919736
40453,1.20,64.6,56.0,2809,0.012153,0.130465,0.739817,194.078394


In [8]:
df_2.columns

Index(['carat', 'depth', 'table', 'price', 'clarity', 'cut', 'color', 'xyz'], dtype='object')

In [9]:
feat_to_scale = ['carat', 'depth', 'table', 'xyz']
other = ['clarity', 'cut', 'color']
# cat_feat = ['city']

In [10]:
scalable = df_2[feat_to_scale]
# categorical = df_2[cat_feat]

In [11]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
# scaler = RobustScaler()
scaled = pd.DataFrame(scaler.fit_transform(scalable), columns=feat_to_scale)

In [12]:
# cat_ohe = pd.get_dummies(categorical, drop_first=True)

In [13]:
df3 = scaled.join(df_2[other])

In [14]:
df3.columns

Index(['carat', 'depth', 'table', 'xyz', 'clarity', 'cut', 'color'], dtype='object')

In [105]:
feat = ['carat', 'depth', 'table', 'clarity', 'cut', 'color']
# feat = ['carat', 'depth', 'table', 'xyz', 'clarity', 'cut', 'color']

target = ['price']

In [106]:
X, y = df3[feat], df_2[target]

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [119]:
# model = LinearRegression()
# hyperparameters = model.get_params()
# print(f'hyperparameters: {hyperparameters}',
#       f'\nmodel: {model}')

# model_rmse = LinearRegression()

# model_rmse = GradientBoostingRegressor(random_state = 42)
model_rmse = RandomForestRegressor(n_estimators= 512, max_depth= 16,  random_state = 42)


model_rmse.fit(X_train, y_train)
prediction = model_rmse.predict(X_test)

hyperparameters = model_rmse.get_params()

rmse = mean_squared_error(y_test, prediction)**0.5

print(type(model_rmse), '\n')
print(rmse, '\n')

C:\Users\magno.MAUDITORS\AppData\Local\Temp\ipykernel_2076\2681890781.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rmse.fit(X_train, y_train)


<class 'sklearn.ensemble._forest.RandomForestRegressor'> 

550.5020437467892 



In [109]:
param_grid = {'n_estimators': [16, 32, 64, 128, 256, 512],
              'max_depth': [2, 4, 8, 16]}

In [147]:
%%time
grid_search = GridSearchCV(RandomForestRegressor(random_state = 42),
                           param_grid,
                           cv=5,
                           verbose=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)

CPU times: total: 156 ms
Wall time: 207 ms


In [113]:
# %%time

# grid_search.fit(X, y)

# print('\n')
# print('Best hyperparameters: ', grid_search.best_params_, '\n')
# print('Best score: ', -grid_search.best_score_, '\n')

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\magnof\AppData\Local\miniconda3\envs\sklearn_env\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)




Best hyperparameters:  {'max_depth': 16, 'n_estimators': 512} 

Best score:  688.3169434200393 

CPU times: total: 1min 24s
Wall time: 7min 7s


In [132]:
df_4 = df_2

In [133]:
df_4.loc[df_4['carat'].between(0, .5, 'left'), 'bin'] = 1.0
df_4.loc[df_4['carat'].between(.5, 1.0, 'left'), 'bin'] = 1.44
df_4.loc[df_4['carat'].between(1.0, 1.5, 'left'), 'bin'] = 2.3
df_4.loc[df_4['carat'].between(1.5, 2.0, 'left'), 'bin'] = 3.21
df_4.loc[df_4['carat'].between(2.0, 3.0, 'left'), 'bin'] = 4.12
df_4.loc[df_4['carat'].between(3.0, 4.0, 'left'), 'bin'] = 5.53
df_4.loc[df_4['carat'].between(4.0, 10.0, 'left'), 'bin'] = 8.39

In [134]:
feat_to_scale_2 = ['carat', 'depth', 'table', 'xyz', 'bin']

In [135]:
scaled_2 = pd.DataFrame(scaler.fit_transform(scalable_2), columns=feat_to_scale_2)

In [136]:
df_5 = scaled_2.join(df_2[other])

In [188]:
feat = ['carat', 'bin', 'depth', 'table', 'clarity', 'cut', 'color']
# feat = ['carat', 'depth', 'table', 'xyz', 'clarity', 'cut', 'color']

target = ['price']

In [189]:
X2, y2 = df5[feat], df_4[target]

In [190]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [140]:
model_rmse = RandomForestRegressor(n_estimators= 512, max_depth= 16,  random_state = 42)

model_rmse.fit(X2_train, y2_train)
prediction = model_rmse.predict(X2_test)

hyperparameters = model_rmse.get_params()

rmse = mean_squared_error(y2_test, prediction)**0.5

print(type(model_rmse), '\n')
print(rmse, '\n')

C:\Users\magno.MAUDITORS\AppData\Local\Temp\ipykernel_2076\635111373.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rmse.fit(X2_train, y2_train)


<class 'sklearn.ensemble._forest.RandomForestRegressor'> 

550.0724567743484 



In [96]:
X2_train

,bin,depth,table,clarity,cut,color
32121,0.078247,0.522222,0.269231,0.323826,0.908900,0.226905
9831,0.242567,0.552778,0.250000,0.524187,0.349414,0.590861
33128,0.078247,0.605556,0.250000,0.748846,0.349414,0.226905
6199,0.000000,0.533333,0.250000,0.524187,0.908900,0.590861
19661,0.000000,0.563889,0.269231,0.323826,0.540255,0.982139
...,...,...,...,...,...,...
6265,0.000000,0.538889,0.230769,0.524187,0.908900,0.590861
11284,0.242567,0.494444,0.288462,0.323826,0.604337,0.853438
38158,0.078247,0.497222,0.192308,0.323826,0.908900,0.739817
860,0.078247,0.475000,0.365385,0.323826,0.540255,0.007570


In [159]:
%%time
## XGBoost regression

regressor=xgb.XGBRegressor(eval_metric='rmsle')
regressor.fit(X2_train, y2_train)
prediction = regressor.predict(X2_test)

rmse = mean_squared_error(y2_test, prediction)**0.5

print(type(regressor), '\n')
print(rmse, '\n')

<class 'xgboost.sklearn.XGBRegressor'> 

557.9035566024004 

CPU times: total: 8.62 s
Wall time: 1.22 s


In [160]:
param_grid = {"max_depth":    [4, 5, 6],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.01, 0.015]}

search = GridSearchCV(regressor, param_grid, cv=5).fit(X2_train, y2_train)
print("The best hyperparameters are ",search.best_params_)

The best hyperparameters are  {'learning_rate': 0.015, 'max_depth': 6, 'n_estimators': 700}


In [194]:
%%time

regressor=xgb.XGBRegressor(learning_rate = 0.015,
                           n_estimators  = 700,
                           max_depth     = 6,
                           eval_metric='rmsle')

regressor.fit(X2_train, y2_train)

prediction = regressor.predict(X2_test)

rmse = mean_squared_error(y2_test, prediction)**0.5

print(type(regressor), '\n')
print(rmse, '\n')

<class 'xgboost.sklearn.XGBRegressor'> 

534.6280457327407 

CPU times: total: 1min 4s
Wall time: 8.9 s


In [192]:
regressor.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': 'rmsle',
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.015,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 6,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 700,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [ ]:
534.6280457327407

,carat,bin,depth,table,clarity,cut,color
32121,0.167442,0.078247,0.522222,0.269231,0.323826,0.908900,0.226905
9831,0.186047,0.242567,0.552778,0.250000,0.524187,0.349414,0.590861
33128,0.069767,0.078247,0.605556,0.250000,0.748846,0.349414,0.226905
6199,0.030233,0.000000,0.533333,0.250000,0.524187,0.908900,0.590861
19661,0.027907,0.000000,0.563889,0.269231,0.323826,0.540255,0.982139
...,...,...,...,...,...,...,...
6265,0.048837,0.000000,0.538889,0.230769,0.524187,0.908900,0.590861
11284,0.200000,0.242567,0.494444,0.288462,0.323826,0.604337,0.853438
38158,0.120930,0.078247,0.497222,0.192308,0.323826,0.908900,0.739817
860,0.123256,0.078247,0.475000,0.365385,0.323826,0.540255,0.007570


,carat,depth,table,clarity,cut,color
32121,0.167442,0.522222,0.269231,0.323826,0.908900,0.226905
9831,0.186047,0.552778,0.250000,0.524187,0.349414,0.590861
33128,0.069767,0.605556,0.250000,0.748846,0.349414,0.226905
6199,0.030233,0.533333,0.250000,0.524187,0.908900,0.590861
19661,0.027907,0.563889,0.269231,0.323826,0.540255,0.982139
...,...,...,...,...,...,...
6265,0.048837,0.538889,0.230769,0.524187,0.908900,0.590861
11284,0.200000,0.494444,0.288462,0.323826,0.604337,0.853438
38158,0.120930,0.497222,0.192308,0.323826,0.908900,0.739817
860,0.123256,0.475000,0.365385,0.323826,0.540255,0.007570


In [156]:
%%time

## lightgbm regression

params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
}


lgb_train = lgb.Dataset(X2_train, y2_train)
lgb_eval = lgb.Dataset(X2_test, y2_test, reference=lgb_train)

model_lightgbm = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=30)


prediction = model_lightgbm.predict(X2_test)

rmse = mean_squared_error(y2_test, prediction)**0.5

print(rmse, '\n')

C:\Users\magnof\AppData\Local\miniconda3\envs\sklearn_env\Lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l2: 1.406e+07	valid_0's l1: 2826.62
Training until validation scores don't improve for 30 rounds
[2]	valid_0's l2: 1.16748e+07	valid_0's l1: 2566.25
[3]	valid_0's l2: 9.73747e+06	valid_0's l1: 2328.95
[4]	valid_0's l2: 8.14783e+06	valid_0's l1: 2114.99
[5]	valid_0's l2: 6.85068e+06	valid_0's l1: 1931.81
[6]	valid_0's l2: 5.7972e+06	valid_0's l1: 1764.61
[7]	valid_0's l2: 4.90881e+06	valid_0's l1: 1610.32
[8]	valid_0's l2: 4.18735e+06	valid_0's l1: 1477.97
[9]	valid_0's l2: 3.58235e+06	valid_0's l1: 1357.63
[10]	valid_0's l2: 3.10534e+06	valid_0's l1: 1255.59
[11]	valid_0's l2: 2.69414e+06	valid_0's l1: 1159.08
[12]	valid_0's l2: 2.35625e+06	valid_0's l1: 1076.21
[13]	valid_0's l2: 2.08278e+06	valid_0's l1: 1001.02
[14]	valid_0's l2: 1.85066e+06	valid_0's l1: 933.13
[15]	valid_0's l2: 1.65376e+06	valid_0's l1: 876.205
[16]	valid_0's l2: 1.49068e+06	valid_0's l1: 826.837
[17]	valid_0's l2: 1.3497e+06	valid_0's l1: 777.75

In [171]:
%%time
## catboost regression

model = CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2)
# Fit model
model.fit(X2_train, y2_train)
# Get predictions
prediction = model.predict(X2_test)

rmse = mean_squared_error(y2_test, prediction)**0.5

print(type(model), '\n')
print(rmse, '\n')

0:	learn: 1811.2473331	total: 6.66ms	remaining: 6.66ms
1:	learn: 1624.9485464	total: 11.6ms	remaining: 0us
<class 'catboost.core.CatBoostRegressor'> 

1679.4245012331621 

CPU times: total: 78.1 ms
Wall time: 78.1 ms


<class 'sklearn.linear_model._base.LinearRegression'> 

1234.1529687253417 

